In [8]:
# libraries used in this project
import torch
from torchvision import datasets
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
from torch import nn,optim
from torch.utils.data import DataLoader
from PIL import Image

In [9]:
#object initialization for transform class
transformation = transforms.Compose([
    transforms.Resize((48,48)),
    transforms.ToTensor(),
    transforms.Grayscale() # Normaize = (original_pixel-mean)/std
])

In [10]:
#train and test data initialization
train_data = datasets.ImageFolder(root=r"D:\Python Projects\Production\Final Project\Emotions Detection\Dataset\dataset\train",transform=transformation)
test_data = datasets.ImageFolder(root= r"D:\Python Projects\Production\Final Project\Emotions Detection\Dataset\dataset\test",transform=transformation)

train_data = DataLoader(train_data,batch_size=64,shuffle=True)
test_data = DataLoader(test_data,batch_size=64,shuffle=True)

In [26]:
#CNN 
class emotion_cnn (nn.Module):
    def __init__(self):
        super(emotion_cnn,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3)#width = (width_in+2*padding-kernel//stride)+1-->46*46
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=1)# downsampled --> 44*44*16
        self.conv2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3)# 42*42*32
        self.input_neurons = 42*42*32
        self.fc1 = nn.Linear(self.input_neurons,128)
        self.fc2 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64,7)
        #self.softmax = nn.LogSoftmax(dim=1)
        #self.softmax = nn.Softmax()
    def forward(self,X):
        X = self.pool1(self.conv1(X))
        X = self.conv2(X)
        X = X.view(-1,self.input_neurons) #Flatten
        X = self.fc1(X)
        X = self.fc2(X)
        X = self.fc4(X)
        #X = self.softmax(X)
        return X

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"The device being used is: {device}")

The device being used is: cpu


In [24]:
# model, loss function, optimizer and Epoch initialization
model = emotion_cnn()
model.to(device)

def weight_initialization(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
            
model.apply(weight_initialization)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.00001, weight_decay= 0.0001)
num_epoch = 200


In [25]:
for epoch in range(1,num_epoch+1):
    optimizer.zero_grad()
    total_error = 0
    for imgs,labels in train_data:
        outputs = model(imgs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        total_error = total_error+loss.item()
    print(f"{epoch}/{num_epoch}, Loss--> {total_error}")

1/200, Loss--> 745.1929994821548
2/200, Loss--> 692.8015923500061
3/200, Loss--> 668.7372939586639
4/200, Loss--> 653.4552992582321
5/200, Loss--> 647.0776482820511
6/200, Loss--> 640.5367410182953
7/200, Loss--> 636.4964466094971
8/200, Loss--> 633.2347677946091
9/200, Loss--> 626.9617183208466
10/200, Loss--> 623.1479606628418
11/200, Loss--> 618.2762442827225
12/200, Loss--> 616.6089909076691
13/200, Loss--> 610.5629742145538
14/200, Loss--> 603.7559517621994
15/200, Loss--> 597.729469537735
16/200, Loss--> 591.152854681015
17/200, Loss--> 587.0175651311874
18/200, Loss--> 585.1080870628357
19/200, Loss--> 579.1814382076263
20/200, Loss--> 576.8662172555923
21/200, Loss--> 577.9613689184189
22/200, Loss--> 572.5727801322937
23/200, Loss--> 565.7712244987488
24/200, Loss--> 562.5916098356247
25/200, Loss--> 560.0915350914001
26/200, Loss--> 557.5242531299591
27/200, Loss--> 555.409304857254
28/200, Loss--> 553.9618277549744
29/200, Loss--> 552.5558257102966
30/200, Loss--> 548.816135

KeyboardInterrupt: 